In [1]:
!pip install seaborn

import seaborn as sns
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import os
from collections import defaultdict

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/"

ROOT = os.path.join(BASE_PATH, DATA_DIR)

FOLDERS = {
    "train": ["anemic_train_roi", "anemic_not_train_roi"],
    "val":   ["anemic_val_roi", "anemic_not_val_roi"],
    "test":  ["anemic_test_roi", "anemic_not_test_roi"]
}

# filename → list of (split, folder_path)
file_locations = defaultdict(list)

# Scan folders
for split, subfolders in FOLDERS.items():
    for sub in subfolders:
        folder_path = os.path.join(ROOT, sub)
        if not os.path.exists(folder_path):
            continue

        for f in os.listdir(folder_path):
            if f.lower().endswith((".png", ".jpg", ".jpeg")):
                file_locations[f].append((split, os.path.join(sub, f)))

# Detect leakage if SAME FILENAME is present in >1 split
leaks_exist = False
print("\nChecking for leakage by FULL FILENAME...\n")

for filename, locations in file_locations.items():
    splits = {loc[0] for loc in locations}
    if len(splits) > 1:
        leaks_exist = True
        print(f"❌ Leakage detected: {filename}")
        for split, path in locations:
            print(f"   → {split}: {path}")
        print()

if not leaks_exist:
    print("✅ No leakage — all filenames are unique to each split.")



Checking for leakage by FULL FILENAME...

✅ No leakage — all filenames are unique to each split.


In [2]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

LEFT_EYE_1 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 200
BATCH_CV = 24
LR_CV = 0.00012
EARLY_STOP_PR = 0.90

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "LEFT_EYE_1_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 44 anemic / 358 total

--- Fold 1 ---
✅ Early stop at epoch 19: P=0.980, R=0.941
Fold 1 → P=0.980, R=0.941

--- Fold 2 ---
Epoch 20/200 Loss: 5.0647
✅ Early stop at epoch 23: P=0.907, R=0.961
Fold 2 → P=0.907, R=0.961

--- Fold 3 ---
✅ Early stop at epoch 16: P=1.000, R=0.961
Fold 3 → P=1.000, R=0.961

--- Fold 4 ---
✅ Early stop at epoch 15: P=0.978, R=0.900
Fold 4 → P=0.978, R=0.900

--- Fold 5 ---
Epoch 20/200 Loss: 8.0869
Epoch 40/200 Loss: 0.7303
Epoch 60/200 Loss: 0.2089
✅ Early stop at epoch 71: P=0.938, R=0.900
Fold 5 → P=0.938, R=0.900
✅ Best fold = 3 | P=1.000, R=0.961

📊 FINAL TEST RESULTS (Original Test Set):
Precision: 1.0000
Recall:    0.9773
F1 score:  0.9885
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 43, 314, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/detailed_predictions_pytorch.csv

✅ Single-eye model finished re

2025-11-16 20:54:57.788911: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 20:54:57.790196: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-16 20:54:57.816754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 20:54:58.335766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-11-16 20:55:00.014415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-16 20:55:00.016141: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/tf_model/assets


...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-16 20:55:04.743024: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-16 20:55:04.743053: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-16 20:55:04.745198: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/tf_model
2025-11-16 20:55:04.775211: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-16 20:55:04.775230: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/tf_model
2025-11-16 20:55:04.806717: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-11-16 20:55:04.807384: I tensorflow/cc/s

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 1.0000
Recall:    0.9773
F1 score:  0.9885
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 43, 314, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 1.0000, R: 0.9773, AUC: 1.0000
TFLite  → P: 1.0000, R: 0.9773, AUC: 1.0000
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_1_eye_original_repro


In [2]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

RIGHT_EYE_2 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 120
BATCH_CV = 16
LR_CV = 0.00003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "RIGHT_EYE_2_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 40 anemic / 369 total

--- Fold 1 ---
✅ Early stop at epoch 14: P=0.922, R=0.904
Fold 1 → P=0.922, R=0.904

--- Fold 2 ---
✅ Early stop at epoch 16: P=0.906, R=0.923
Fold 2 → P=0.906, R=0.923

--- Fold 3 ---
✅ Early stop at epoch 15: P=0.939, R=0.902
Fold 3 → P=0.939, R=0.902

--- Fold 4 ---
✅ Early stop at epoch 18: P=0.961, R=0.961
Fold 4 → P=0.961, R=0.961

--- Fold 5 ---
✅ Early stop at epoch 14: P=0.980, R=0.961
Fold 5 → P=0.980, R=0.961
✅ Best fold = 5 | P=0.980, R=0.961

📊 FINAL TEST RESULTS (Original Test Set):
Precision: 1.0000
Recall:    0.9750
F1 score:  0.9873
Accuracy:  0.9973
AUC:       1.0000
TP, TN, FP, FN: 39, 329, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/detailed_predictions_pytorch.csv

✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.


2025-11-16 22:07:58.125813: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 22:07:58.127257: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-16 22:07:58.156945: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 22:07:58.690282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-11-16 22:08:00.486126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-16 22:08:00.487969: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/tf_model/assets


...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-16 22:08:05.711881: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-16 22:08:05.711921: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-16 22:08:05.714150: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/tf_model
2025-11-16 22:08:05.757335: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-16 22:08:05.757360: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/tf_model
2025-11-16 22:08:05.792591: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-11-16 22:08:05.793436: I tensorflow/cc

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 1.0000
Recall:    0.9750
F1 score:  0.9873
Accuracy:  0.9973
AUC:       1.0000
TP, TN, FP, FN: 39, 329, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 1.0000, R: 0.9750, AUC: 1.0000
TFLite  → P: 1.0000, R: 0.9750, AUC: 1.0000
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_2_eye_original_repro


In [3]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

LEFT_EYE_2 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 25
BATCH_CV = 8
LR_CV = 0.00028

EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "LEFT_EYE_2_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 42 anemic / 361 total

--- Fold 1 ---
✅ Early stop at epoch 9: P=0.939, R=0.902
Fold 1 → P=0.939, R=0.902

--- Fold 2 ---
✅ Early stop at epoch 7: P=0.962, R=0.980
Fold 2 → P=0.962, R=0.980

--- Fold 3 ---
✅ Early stop at epoch 3: P=0.926, R=0.980
Fold 3 → P=0.926, R=0.980

--- Fold 4 ---
✅ Early stop at epoch 4: P=0.960, R=0.941
Fold 4 → P=0.960, R=0.941

--- Fold 5 ---
✅ Early stop at epoch 7: P=0.980, R=0.980
Fold 5 → P=0.980, R=0.980
✅ Best fold = 5 | P=0.980, R=0.980

📊 FINAL TEST RESULTS (Original Test Set):
Precision: 1.0000
Recall:    0.9762
F1 score:  0.9880
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 41, 319, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/detailed_predictions_pytorch.csv

✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.

--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to 

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-16 22:25:10.571445: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-16 22:25:10.571478: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-16 22:25:10.573253: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/tf_model
2025-11-16 22:25:10.605275: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-16 22:25:10.605295: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/tf_model
2025-11-16 22:25:10.629608: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-16 22:25:10.691337: I tensorflow/cc/saved_model/loader.cc:215] Running

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 1.0000
Recall:    0.9762
F1 score:  0.9880
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 41, 319, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_2_eye_original_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 1.0000, R: 0.9762, AUC: 1.0000
TFLite  → P: 1.0000, R: 0.9762, AUC: 1.0000
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and compreh

In [4]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

RIGHT_EYE_3 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 120
BATCH_CV = 32
LR_CV = 0.000003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90

BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "RIGHT_EYE_3_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)


# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 40 anemic / 368 total

--- Fold 1 ---
Epoch 20/120 Loss: 17.4501
Epoch 40/120 Loss: 7.7666
✅ Early stop at epoch 59: P=0.958, R=0.902
Fold 1 → P=0.958, R=0.902

--- Fold 2 ---
Epoch 20/120 Loss: 17.7057
Epoch 40/120 Loss: 12.1958
✅ Early stop at epoch 53: P=0.902, R=0.902
Fold 2 → P=0.902, R=0.902

--- Fold 3 ---
Epoch 20/120 Loss: 17.7456
Epoch 40/120 Loss: 13.1826
Epoch 60/120 Loss: 7.5461
Epoch 80/120 Loss: 3.9320
Epoch 100/120 Loss: 2.0900
✅ Early stop at epoch 113: P=0.922, R=0.922
Fold 3 → P=0.922, R=0.922

--- Fold 4 ---
Epoch 20/120 Loss: 17.4451
Epoch 40/120 Loss: 12.7434
Epoch 60/120 Loss: 7.0670
Epoch 80/120 Loss: 3.5584
Epoch 100/120 Loss: 1.8323
Epoch 120/120 Loss: 0.9650
Fold 4 → P=1.000, R=0.804

--- Fold 5 ---
Epoch 20/120 Loss: 17.3121
Epoch 40/120 Loss: 12.2740
Epoch 60/120 Loss: 6.6113
Epoch 80/120 Loss: 3.2226
Epoch 100/120 Loss: 1.7944
✅ Early stop at epoch 113: P=0.958, R=0.902
Fold 5 → P=0.958, R=0.902
✅ Best fo

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-17 01:46:06.240110: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 01:46:06.240144: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 01:46:06.242346: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/tf_model
2025-11-17 01:46:06.273307: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 01:46:06.273331: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/tf_model
2025-11-17 01:46:06.303686: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 01:46:06.366454: I tensorflow/cc/saved_model/loader.cc:215] Runni

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 1.0000
Recall:    0.8750
F1 score:  0.9333
Accuracy:  0.9864
AUC:       0.9987
TP, TN, FP, FN: 35, 328, 0, 5
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_3_eye_original_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 1.0000, R: 0.8750, AUC: 0.9987
TFLite  → P: 1.0000, R: 0.8750, AUC: 0.9987
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and compr

In [5]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

LEFT_EYE_3 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 20
BATCH_CV = 16
LR_CV = 0.00028

EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "LEFT_EYE_3_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)
# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 43 anemic / 361 total

--- Fold 1 ---
✅ Early stop at epoch 3: P=0.942, R=0.980
Fold 1 → P=0.942, R=0.980

--- Fold 2 ---
✅ Early stop at epoch 2: P=1.000, R=0.980
Fold 2 → P=1.000, R=0.980

--- Fold 3 ---
✅ Early stop at epoch 3: P=0.962, R=1.000
Fold 3 → P=0.962, R=1.000

--- Fold 4 ---
✅ Early stop at epoch 4: P=1.000, R=0.960
Fold 4 → P=1.000, R=0.960

--- Fold 5 ---
✅ Early stop at epoch 2: P=0.980, R=1.000
Fold 5 → P=0.980, R=1.000
✅ Best fold = 5 | P=0.980, R=1.000

📊 FINAL TEST RESULTS (Original Test Set):
Precision: 1.0000
Recall:    0.9767
F1 score:  0.9882
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 42, 318, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/detailed_predictions_pytorch.csv

✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.

--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to 

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-11-17 01:55:54.387703: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 01:55:54.387739: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 01:55:54.389450: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/tf_model
2025-11-17 01:55:54.423972: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 01:55:54.424005: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/tf_model
2025-11-17 01:55:54.435965: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 01:55:54.461155: I tensorflow/cc/saved_model/loader.cc:215] Running

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...
🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 1.0000
Recall:    0.9767
F1 score:  0.9882
Accuracy:  0.9972
AUC:       1.0000
TP, TN, FP, FN: 42, 318, 0, 1
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/LEFT_EYE_3_eye_original_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 1.0000, R: 0.9767, AUC: 1.0000
TFLite  → P: 1.0000, R: 0.9767, AUC: 1.0000
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and compreh

In [2]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

RIGHT_EYE_1 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 1024
EPOCHS_CV = 220
BATCH_CV = 14
LR_CV = 0.00003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90


BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "RIGHT_EYE_1_eye_original_repro_1024")
os.makedirs(OUTPUT_DIR, exist_ok=True)
# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 43 anemic / 365 total

--- Fold 1 ---
Epoch 20/220 Loss: 11.2012
Epoch 40/220 Loss: 1.9405
Epoch 60/220 Loss: 1.0357
Epoch 80/220 Loss: 1.3969
Epoch 100/220 Loss: 0.6448
Epoch 120/220 Loss: 1.7892
Epoch 140/220 Loss: 0.0969
Epoch 160/220 Loss: 0.2910
Epoch 180/220 Loss: 0.1500
Epoch 200/220 Loss: 0.4321
Epoch 220/220 Loss: 0.0362
Fold 1 → P=0.605, R=0.510

--- Fold 2 ---
Epoch 20/220 Loss: 10.9389
Epoch 40/220 Loss: 2.5949
Epoch 60/220 Loss: 1.4460
Epoch 80/220 Loss: 0.8656
Epoch 100/220 Loss: 1.3135
Epoch 120/220 Loss: 0.9017
Epoch 140/220 Loss: 2.0926
Epoch 160/220 Loss: 0.0310
Epoch 180/220 Loss: 0.6605
Epoch 200/220 Loss: 0.0778
Epoch 220/220 Loss: 0.1675
Fold 2 → P=0.585, R=0.471

--- Fold 3 ---
Epoch 20/220 Loss: 11.9621
Epoch 40/220 Loss: 1.8696
Epoch 60/220 Loss: 1.8537
Epoch 80/220 Loss: 0.6768
Epoch 100/220 Loss: 0.7576
Epoch 120/220 Loss: 1.0454
Epoch 140/220 Loss: 0.4913
Epoch 160/220 Loss: 0.0689
Epoch 180/220 Loss: 0.280

2025-12-03 00:11:52.786519: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-03 00:11:52.787907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 00:11:52.815424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 00:11:53.477639: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-12-03 00:11:55.751295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-03 00:11:55.752917: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/tf_model/assets


...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-12-03 00:12:00.948947: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-03 00:12:00.948985: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-03 00:12:00.952196: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/tf_model
2025-12-03 00:12:00.989071: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-12-03 00:12:00.989101: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/tf_model
2025-12-03 00:12:01.023099: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-12-03 00:12:01.024059: I ten

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [   1    3 1024 1024]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 0.6500
Recall:    0.3023
F1 score:  0.4127
Accuracy:  0.8986
AUC:       0.8706
TP, TN, FP, FN: 13, 315, 7, 30
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 0.6500, R: 0.3023, AUC: 0.8706
TFLite  → P: 0.6500, R: 0.3023, AUC: 0.8706
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro_1024


In [1]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST + TFLite Re-eval)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient
- Early stop if P & R >= 0.90 on validation
- TFLite conversion + re-evaluation on same test set with auto-detected input size
- Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
- Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison

RIGHT_EYE_1 (Chronological Split)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 300
BATCH_CV = 14
LR_CV = 0.00003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90


BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "RIGHT_EYE_1_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)
# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING WITH FILENAMES
# =========================
def load_images_with_filenames(folder, label):
    imgs, lbls, filenames = [], [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
                    filenames.append(f)
    return imgs, lbls, filenames

train_imgs, train_lbls, train_filenames = [], [], []
val_imgs, val_lbls, val_filenames = [], [], []
test_imgs, test_lbls, test_filenames = [], [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    train_imgs+=i; train_lbls+=l; train_filenames+=f
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    val_imgs+=i; val_lbls+=l; val_filenames+=f
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l,f = load_images_with_filenames(folder,label)
    test_imgs+=i; test_lbls+=l; test_filenames+=f

print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    preds, probs, labels_all = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (p > 0.5).astype(int)
        preds.extend(pred.tolist())
        probs.extend(p.tolist())
        labels_all.extend(labels.cpu().numpy().flatten().tolist())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0, labels_all, probs, all_filenames, preds
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn, labels_all, probs, all_filenames, preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# TRAINING LOOP
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    X = train_imgs
    y = train_lbls
    filenames = train_filenames
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        train_subset_filenames = [filenames[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        val_subset_filenames = [filenames[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _, _, _, _, _ = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        val_metrics = evaluate_with_predictions(model, vl_loader, val_subset_filenames)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # Final test evaluation
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    test_metrics = evaluate_with_predictions(model, test_loader, test_filenames)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    
    # Save detailed predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(OUTPUT_DIR, "detailed_predictions_pytorch.csv")
    )
    
    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(OUTPUT_DIR, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(OUTPUT_DIR, "confusion_matrix_pytorch.png"))
    
    return model

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(model, output_dir, resolution):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    try:
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            # No dynamic_axes → fixes input size
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow failed: {e}")
        return

    # TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        tflite_model = converter.convert()
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# TFLITE EVALUATION (AUTO-DETECT INPUT SIZE) WITH PREDICTIONS
# =========================
def evaluate_tflite_on_test_with_predictions(tflite_path, test_imgs, test_lbls, test_filenames):
    import tensorflow as tf
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
    import numpy as np
    from PIL import Image
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    expected_shape = input_details[0]['shape']
    print(f"🔍 TFLite model input shape: {expected_shape}")

    if len(expected_shape) != 4:
        raise ValueError(f"Expected 4D input, got {expected_shape}")
    
    batch = expected_shape[0]
    assert batch == 1, "Batch size must be 1"

    # Detect layout: NCHW if shape[1] == 3, NHWC if shape[3] == 3
    if expected_shape[1] == 3 and expected_shape[3] != 3:
        layout = 'NCHW'
        _, _, h, w = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NCHW")
    elif expected_shape[3] == 3 and expected_shape[1] != 3:
        layout = 'NHWC'
        _, h, w, _ = expected_shape
        resize_h, resize_w = int(h), int(w)
        print(f"   ➤ Detected layout: NHWC")
    else:
        # Fallback: assume NHWC if last dim is 3
        if expected_shape[-1] == 3:
            layout = 'NHWC'
            _, h, w, _ = expected_shape
            resize_h, resize_w = int(h), int(w)
        elif expected_shape[1] == 3:
            layout = 'NCHW'
            _, _, h, w = expected_shape
            resize_h, resize_w = int(h), int(w)
        else:
            raise ValueError(f"Cannot determine layout from shape {expected_shape}")

    preds, probs, labels_all = [], [], []

    # Use PIL + torchvision to EXACTLY match PyTorch preprocessing
    def preprocess_pil_style(img_rgb, target_size):
        """
        Reproduce: 
          transforms.ToPILImage() → Resize → ToTensor → Normalize
        """
        # img_rgb: numpy array (H, W, C), uint8, RGB
        pil_img = Image.fromarray(img_rgb)
        # Resize with PIL BILINEAR (same as torchvision)
        resized_pil = pil_img.resize((target_size[1], target_size[0]), Image.BILINEAR)  # (W, H)
        # ToTensor: (H, W, C) uint8 → (C, H, W) float32 [0,1]
        tensor = to_tensor(resized_pil)  # shape: (C, H, W)
        # Normalize with ImageNet stats
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()  # (C, H, W), float32

    for img, label in zip(test_imgs, test_lbls):
        # img is RGB numpy array (H, W, C), uint8 — same as loaded by cv2.cvtColor(..., cv2.COLOR_BGR2RGB)
        img_norm_nchw = preprocess_pil_style(img, (resize_h, resize_w))  # (C, H, W)

        if layout == 'NCHW':
            input_data = np.expand_dims(img_norm_nchw, axis=0)  # (1, C, H, W)
        else:  # NHWC
            img_norm_nhwc = np.transpose(img_norm_nchw, (1, 2, 0))  # (H, W, C)
            input_data = np.expand_dims(img_norm_nhwc, axis=0)  # (1, H, W, C)

        input_data = input_data.astype(input_details[0]['dtype'])

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1]
        logit = output[0][0]
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        preds.append(pred)
        probs.append(prob)
        labels_all.append(label)

    if len(set(labels_all)) < 2:
        print("⚠️ Only one class in test set!")
        return None

    P, R, F1, _ = precision_recall_fscore_support(labels_all, preds, average='binary')
    acc = accuracy_score(labels_all, preds)
    auc = roc_auc_score(labels_all, probs)
    tn, fp, fn, tp = confusion_matrix(labels_all, preds, labels=[0, 1]).ravel()

    return P, R, F1, acc, auc, tp, tn, fp, fn, labels_all, probs, test_filenames, preds

# =========================
# MAIN EXECUTION
# =========================
if __name__ == "__main__":
    # Train and evaluate
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    # Re-evaluate TFLite on same test set
    print("\n🔍 Loading TFLite model and re-evaluating on original test set...")
    tflite_file = os.path.join(OUTPUT_DIR, "single_eye_resnet18.tflite")
    if not os.path.exists(tflite_file):
        raise FileNotFoundError(f"TFLite model not found at {tflite_file}")

    tflite_metrics = evaluate_tflite_on_test_with_predictions(tflite_file, test_imgs, test_lbls, test_filenames)

    if tflite_metrics:
        P, R, F1, acc, auc, tp, tn, fp, fn, tflite_labels, tflite_probs, tflite_filenames, tflite_preds = tflite_metrics
        print("\n📊 TFLITE TEST RESULTS (Same test set):")
        print(f"Precision: {P:.4f}")
        print(f"Recall:    {R:.4f}")
        print(f"F1 score:  {F1:.4f}")
        print(f"Accuracy:  {acc:.4f}")
        print(f"AUC:       {auc:.4f}")
        print(f"TP, TN, FP, FN: {int(tp)}, {int(tn)}, {int(fp)}, {int(fn)}")

        # Save detailed TFLite predictions to CSV
        save_predictions_to_csv(
            tflite_filenames,
            tflite_labels,
            tflite_preds,
            tflite_probs,
            os.path.join(OUTPUT_DIR, "detailed_predictions_tflite.csv")
        )

        # Plot ROC curve and confusion matrix for TFLite model
        plot_roc_curve(tflite_labels, tflite_probs, 
                       "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                       os.path.join(OUTPUT_DIR, "roc_curve_tflite.png"))
        plot_confusion_matrix(tflite_labels, 
                              tflite_preds, 
                              "Confusion Matrix - TFLite Model", 
                              os.path.join(OUTPUT_DIR, "confusion_matrix_tflite.png"))

        test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
        if os.path.exists(test_results_path):
            orig = pd.read_csv(test_results_path).iloc[0]
            print("\n🔍 Comparing with original PyTorch test results:")
            print(f"PyTorch → P: {orig['Test_Precision']:.4f}, R: {orig['Test_Recall']:.4f}, AUC: {orig['Test_AUC']:.4f}")
            print(f"TFLite  → P: {P:.4f}, R: {R:.4f}, AUC: {auc:.4f}")
            
            # Create metrics comparison plot
            plot_metrics_comparison(orig.to_dict(), tflite_metrics, 
                                   os.path.join(OUTPUT_DIR, "metrics_comparison.png"))
            
            tol = 1e-3
            p_ok = abs(P - orig['Test_Precision']) < tol
            r_ok = abs(R - orig['Test_Recall']) < tol
            auc_ok = abs(auc - orig['Test_AUC']) < tol
            
            if p_ok and r_ok and auc_ok:
                print("✅ TFLite results match PyTorch within tolerance (1e-3).")
            else:
                print("⚠️ TFLite results differ from PyTorch (check normalization or export).")
        else:
            print("⚠️ Original test results not found for comparison.")
    else:
        print("❌ TFLite evaluation failed.")

    print("\n✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!")
    print(f"✅ Detailed prediction CSVs and plots saved to: {OUTPUT_DIR}")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TEST: 43 anemic / 365 total

--- Fold 1 ---
Epoch 20/300 Loss: 7.3127
Epoch 40/300 Loss: 2.7326
Epoch 60/300 Loss: 1.6939
Epoch 80/300 Loss: 1.4495
Epoch 100/300 Loss: 0.5191
Epoch 120/300 Loss: 0.1237
Epoch 140/300 Loss: 0.2180
Epoch 160/300 Loss: 0.2899
Epoch 180/300 Loss: 0.1262
Epoch 200/300 Loss: 0.0526
Epoch 220/300 Loss: 1.1068
Epoch 240/300 Loss: 0.7019
Epoch 260/300 Loss: 0.0751
Epoch 280/300 Loss: 0.9214
Epoch 300/300 Loss: 0.0118
Fold 1 → P=0.732, R=0.588

--- Fold 2 ---
Epoch 20/300 Loss: 6.8190
Epoch 40/300 Loss: 3.4321
Epoch 60/300 Loss: 2.6696
Epoch 80/300 Loss: 0.5608
Epoch 100/300 Loss: 0.6304
Epoch 120/300 Loss: 0.8943
Epoch 140/300 Loss: 0.1339
Epoch 160/300 Loss: 0.1590
Epoch 180/300 Loss: 0.4625
Epoch 200/300 Loss: 2.7557
Epoch 220/300 Loss: 1.1742
Epoch 240/300 Loss: 4.7815
Epoch 260/300 Loss: 0.9910
Epoch 280/300 Loss: 0.0109
Epoch 300/300 Loss: 3.1628
Fold 2 → P=0.222, R=0.980

--- Fold 3 ---
Epoch 20/300 Loss: 8.235

2025-12-01 16:25:27.319722: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 16:25:27.321017: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-01 16:25:27.347819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 16:25:27.865682: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-12-01 16:25:29.655368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 16:25:29.657007: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_19_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

...

   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-12-01 16:25:34.059185: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-01 16:25:34.059215: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-01 16:25:34.061487: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro/tf_model
2025-12-01 16:25:34.094169: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-12-01 16:25:34.094188: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro/tf_model
2025-12-01 16:25:34.126070: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-12-01 16:25:34.126753: I tensorflow/cc

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.

🔍 Loading TFLite model and re-evaluating on original test set...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🔍 TFLite model input shape: [  1   3 780 780]
   ➤ Detected layout: NCHW

📊 TFLITE TEST RESULTS (Same test set):
Precision: 0.4762
Recall:    0.2326
F1 score:  0.3125
Accuracy:  0.8795
AUC:       0.8337
TP, TN, FP, FN: 10, 311, 11, 33
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro/detailed_predictions_tflite.csv

🔍 Comparing with original PyTorch test results:
PyTorch → P: 0.4762, R: 0.2326, AUC: 0.8337
TFLite  → P: 0.4762, R: 0.2326, AUC: 0.8337
✅ TFLite results match PyTorch within tolerance (1e-3).

✅ Analysis complete: Original chronological test set evaluated with detailed CSV predictions and comprehensive plots!
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/RIGHT_EYE_1_eye_original_repro


In [7]:
# -*- coding: utf-8 -*-
"""
Reproducible Tri-right-Eye ResNet18 with SHARED BACKBONE & TFLite Verification
----------------------------------------------------------------------------------
✅ Shared ResNet18 across 3 inputs → ~45 MB TFLite
✅ TFLite export with SELECT_TF_OPS (GELU via FlexDelegate)
✅ Re-evaluates .tflite file and compares results with PyTorch
✅ Ensures no silent divergence between frameworks
✅ Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
✅ Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
"""

import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224  # Only one resolution used now
EPOCHS_CV = 150
BATCH_CV = 28
LR_CV = 0.00022

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri_right_eye_hb_90_repro_bestfold_only_shared")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

train_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_right(dirs_map):
    suffixes = {'right1': '_right_eye_1.png', 'right2': '_right_eye_2.png', 'right3': '_right_eye_3.png'}
    bases_sets = []
    for k in ['right1', 'right2', 'right3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_tri_images_by_bases_with_filenames(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': [], 'filenames': []}
    key_map = {'r1': ('right1', '_right_eye_1.png'), 'r2': ('right2', '_right_eye_2.png'), 'r3': ('right3', '_right_eye_3.png')}
    for b in bases:
        imgs, failed = {}, False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if not failed:
            out['r1'].append(imgs['r1'])
            out['r2'].append(imgs['r2'])
            out['r3'].append(imgs['r3'])
            out['filenames'].append(b)  # Use base name as identifier
    return out

def prepare_dataset_right_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_right(anemic_dirs)
    bases_n = common_bases_right(non_dirs)
    imgs_a = load_tri_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_tri_images_by_bases_with_filenames(non_dirs, bases_n)
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_right.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in ['right1','right2','right3']:
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================

print_dir_stats("TEST  anemic (right)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (right)", test_dirs_non)

train_data = prepare_dataset_right_with_filenames(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_right_with_filenames(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_right_with_filenames(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-right-eye TRAIN samples found.")

# =========================
# DATASET
# =========================
class TrirightDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data['label'])
    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
    )

# =========================
# MODEL: SHARED RESNET18 BACKBONE
# =========================
class TriResNetright(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()

        self.fusion = nn.Sequential(
            nn.Linear(3 * 512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, x1, x2, x3):
        f1 = self.backbone(x1)
        f2 = self.backbone(x2)
        f3 = self.backbone(x3)
        x = torch.cat([f1, f2, f3], dim=1)
        return self.fusion(x)

# =========================
# EVALUATION (PyTorch) WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        x1, x2, x3 = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(x1, x2, x3)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(all_labels) == 0 or len(set(all_labels)) < 2:
        return [float('nan')] * 9 + [all_labels, all_probs, all_filenames, all_preds]

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int, tflite_filename: str):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    best_model.eval().to('cpu')
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution) for _ in range(3))
    onnx_path = os.path.join(output_dir, "tri_right_model.onnx")
    tf_path = os.path.join(output_dir, "tri_right_tf_model")
    tflite_path = os.path.join(output_dir, tflite_filename)

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1: PyTorch -> ONNX
    print("1. Converting to ONNX...")
    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input1', 'input2', 'input3'],
            output_names=['output']
        )
        print(f"   ✅ ONNX saved: {onnx_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 2: ONNX -> TensorFlow
    print("2. ONNX -> TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        if os.path.exists(tf_path):
            import shutil
            shutil.rmtree(tf_path)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TF SavedModel saved: {tf_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 3: TF -> TFLite with SELECT_TF_OPS
    print("3. TF -> TFLite (SELECT_TF_OPS)...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"   ✅ TFLite saved: {tflite_path} ({size_mb:.2f} MB)")
        return tflite_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[1] == 3:  # [B,C,H,W]
            layout = 'NCHW'
        else:  # [B,H,W,C]
            layout = 'NHWC'

    resize_h, resize_w = int(first_shape[1] if layout == 'NHWC' else first_shape[2]), \
                         int(first_shape[2] if layout == 'NHWC' else first_shape[3])

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data['r1'][i], test_data['r2'][i], test_data['r3'][i]]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN TRAINING LOOP
# =========================
if __name__ == "__main__":
    resolution = RESOLUTION
    results = []
    cv_index_records = []

    print(f"\n===== Processing right resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(train_data['label'])

    fold = 1
    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- right Fold {fold} ---")
        cv_index_records.append({"fold": fold, "train_indices": train_idx.tolist(), "val_indices": val_idx.tolist()})

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in train_data.items()}

        train_loader = make_loader(TrirightDataset(train_subset, train_transform), BATCH_CV, True)
        val_loader   = make_loader(TrirightDataset(val_subset,   test_transform),  BATCH_CV, False)

        model = TriResNetright().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                x1, x2, x3 = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(x1, x2, x3)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{EPOCHS_CV}] Loss: {total_loss:.6f}")

        val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
        result_row = {
            'EyeSet': 'right',
            'Resolution': resolution,
            'Fold': fold,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(result_row)
        print(results)
        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"right_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict()}, fold_path)

        fold += 1

    # Save CV results
    pd.DataFrame(results).to_csv(os.path.join(output_dir, "right_val_cross_validation_results.csv"), index=False)
    with open(os.path.join(output_dir, "right_cv_indices.json"), "w") as f:
        json.dump(cv_index_records, f, indent=2)

    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = df[(df['Val_Precision'] >= 0.90) & (df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1'], ascending=False).iloc[0] if len(candidates) > 0 else \
           df.sort_values(['minPR','Val_F1'], ascending=False).iloc[0]
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best model
    ckpt_path = os.path.join(output_dir, f"right_cv_fold_{best_fold}_res{resolution}.pt")
    best_model = TriResNetright().to(device)
    state = torch.load(ckpt_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Test evaluation (PyTorch)
    test_loader = make_loader(TrirightDataset(test_data, test_transform), BATCH_CV, False)
    test_metrics = evaluate_with_predictions(best_model, test_loader, test_data['filenames'])

    test_results_df = pd.DataFrame([{
        'ChosenFold': best_fold,
        'Test_Precision': test_metrics[0],
        'Test_Recall': test_metrics[1],
        'Test_F1': test_metrics[2],
        'Test_Accuracy': test_metrics[3],
        'Test_AUC': test_metrics[4],
        'Test_TP': test_metrics[5],
        'Test_TN': test_metrics[6],
        'Test_FP': test_metrics[7],
        'Test_FN': test_metrics[8]
    }])
    test_results_df.to_csv(os.path.join(output_dir, "right_bestfold_test_results.csv"), index=False)

    print("\n📊 TEST Results (Shared Backbone):")
    print(test_results_df.to_string(index=False))

    # Save detailed PyTorch predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(output_dir, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(output_dir, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(output_dir, "confusion_matrix_pytorch.png"))

    # Convert to TFLite
    tflite_filename = "tri_right_eye_resnet18_shared.tflite"
    tflite_path = convert_to_tflite(best_model.cpu(), output_dir, resolution, tflite_filename)

    if tflite_path:
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"\n🎉 SUCCESS! Final TFLite model size: {size_mb:.2f} MB")
        print(f"📍 Path: {tflite_path}")

        # --- 🔍 Re-evaluate TFLite model ---
        print("\n🔍 Re-evaluating TFLite model on test set...")
        try:
            tflite_metrics = evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution)
            tflite_results_df = pd.DataFrame([{
                'Source': 'TFLite',
                'Test_Precision': tflite_metrics[0],
                'Test_Recall': tflite_metrics[1],
                'Test_F1': tflite_metrics[2],
                'Test_Accuracy': tflite_metrics[3],
                'Test_AUC': tflite_metrics[4],
                'Test_TP': tflite_metrics[5],
                'Test_TN': tflite_metrics[6],
                'Test_FP': tflite_metrics[7],
                'Test_FN': tflite_metrics[8]
            }])

            combined = pd.concat([
                test_results_df.assign(Source='PyTorch'),
                tflite_results_df
            ], ignore_index=True)

            print("\n📊 COMPARISON: PyTorch vs TFLite")
            print(combined.to_string(index=False))
            combined.to_csv(os.path.join(output_dir, "pytorch_vs_tflite_comparison.csv"), index=False)

            # Save detailed TFLite predictions to CSV
            save_predictions_to_csv(
                tflite_metrics[11],  # filenames
                tflite_metrics[9],   # true labels
                tflite_metrics[12],  # pred labels
                tflite_metrics[10],  # pred probs
                os.path.join(output_dir, "detailed_predictions_tflite.csv")
            )

            # Plot ROC curve and confusion matrix for TFLite model
            plot_roc_curve(tflite_metrics[9], tflite_metrics[10], 
                           "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                           os.path.join(output_dir, "roc_curve_tflite.png"))
            plot_confusion_matrix(tflite_metrics[9], 
                                  tflite_metrics[12], 
                                  "Confusion Matrix - TFLite Model", 
                                  os.path.join(output_dir, "confusion_matrix_tflite.png"))

            # Create metrics comparison plot
            plot_metrics_comparison(test_results_df.iloc[0].to_dict(), tflite_metrics, 
                                   os.path.join(output_dir, "metrics_comparison.png"))

            tol = 1e-3
            if (abs(tflite_metrics[2] - test_metrics[2]) < tol and
                abs(tflite_metrics[4] - test_metrics[4]) < tol):
                print("✅ TFLite results MATCH PyTorch within tolerance.")
            else:
                print("⚠️ WARNING: TFLite results differ significantly from PyTorch!")
        except Exception as e:
            print(f"❌ TFLite evaluation failed: {e}")
    else:
        print("❌ TFLite conversion failed.")

    print("\n✅ Pipeline completed. Model size reduced to ~45 MB via shared backbone.")
    print(f"✅ Detailed prediction CSVs and plots saved to: {output_dir}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (right)
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40
right3  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40

📂 TEST  non-anemic (right)
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/ | files=322
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemi

INFO:absl:Function `__call__` contains input name(s) input, x, y with unsupported characters which will be renamed to onnx_tf_prefix_identity_49_input, transpose_187_x, onnx_tf_prefix__fusion_fusion_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_tf_model/fingerprint.pb


   ✅ TF SavedModel saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_tf_model
3. TF -> TFLite (SELECT_TF_OPS)...


2025-11-17 16:37:02.592786: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 16:37:02.592820: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 16:37:02.595167: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_tf_model
2025-11-17 16:37:02.631794: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 16:37:02.631823: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_tf_model
2025-11-17 16:37:02.655531: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 16:37:02.7418

   ✅ TFLite saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_eye_resnet18_shared.tflite (45.92 MB)

🎉 SUCCESS! Final TFLite model size: 45.92 MB
📍 Path: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/tri_right_eye_resnet18_shared.tflite

🔍 Re-evaluating TFLite model on test set...


INFO: Created TensorFlow Lite delegate for select TF ops.
2025-11-17 16:37:04.209318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-17 16:37:04.210806: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO: TfLiteFlexDelegate delegate: 2 nodes delegated out of 285 nodes with 2 partitions.



🔍 TFLite input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0']
   ➤ Detected layout: NCHW, size: 224x224

📊 COMPARISON: PyTorch vs TFLite
 ChosenFold  Test_Precision  Test_Recall  Test_F1  Test_Accuracy  Test_AUC  Test_TP  Test_TN  Test_FP  Test_FN  Source
        4.0             1.0     0.911765 0.953846       0.990566       1.0       31      284        0        3 PyTorch
        NaN             1.0     0.911765 0.953846       0.990566       1.0       31      284        0        3  TFLite
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_bestfold_only_shared/detailed_predictions_tflite.csv
✅ TFLite results MATCH PyTorch within tolerance.

✅ Pipeline completed. Model size reduced to ~45 MB via shared backbone.
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye_hb_90_repro_b

In [8]:
# -*- coding: utf-8 -*-
"""
Reproducible Tri-left-Eye ResNet18 with SHARED BACKBONE & TFLite Verification
----------------------------------------------------------------------------------
✅ Shared ResNet18 across 3 inputs → ~45 MB TFLite
✅ TFLite export with SELECT_TF_OPS (GELU via FlexDelegate)
✅ Re-evaluates .tflite file and compares results with PyTorch
✅ Ensures no silent divergence between frameworks
✅ Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
✅ Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
"""

import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================

SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224
EPOCHS_CV = 120
BATCH_CV = 24
LR_CV = 0.00017

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri_left_eye_hb_90_repro_bestfold_only_shared")
os.makedirs(output_dir, exist_ok=True)




def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

train_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_left(dirs_map):
    suffixes = {'left1': '_left_eye_1.png', 'left2': '_left_eye_2.png', 'left3': '_left_eye_3.png'}
    bases_sets = []
    for k in ['left1', 'left2', 'left3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_tri_images_by_bases_with_filenames(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': [], 'filenames': []}
    key_map = {'r1': ('left1', '_left_eye_1.png'), 'r2': ('left2', '_left_eye_2.png'), 'r3': ('left3', '_left_eye_3.png')}
    for b in bases:
        imgs, failed = {}, False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if not failed:
            out['r1'].append(imgs['r1'])
            out['r2'].append(imgs['r2'])
            out['r3'].append(imgs['r3'])
            out['filenames'].append(b)  # Use base name as identifier
    return out

def prepare_dataset_left_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_left(anemic_dirs)
    bases_n = common_bases_left(non_dirs)
    imgs_a = load_tri_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_tri_images_by_bases_with_filenames(non_dirs, bases_n)
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_left.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in ['left1','left2','left3']:
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================

print_dir_stats("TEST  anemic (left)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (left)", test_dirs_non)

train_data = prepare_dataset_left_with_filenames(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_left_with_filenames(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_left_with_filenames(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-left-eye TRAIN samples found.")

# =========================
# DATASET
# =========================
class TrileftDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data['label'])
    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
    )

# =========================
# MODEL: SHARED RESNET18 BACKBONE
# =========================
class TriResNetleft(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()

        self.fusion = nn.Sequential(
            nn.Linear(3 * 512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, x1, x2, x3):
        f1 = self.backbone(x1)
        f2 = self.backbone(x2)
        f3 = self.backbone(x3)
        x = torch.cat([f1, f2, f3], dim=1)
        return self.fusion(x)

# =========================
# EVALUATION (PyTorch) WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        x1, x2, x3 = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(x1, x2, x3)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(all_labels) == 0 or len(set(all_labels)) < 2:
        return [float('nan')] * 9 + [all_labels, all_probs, all_filenames, all_preds]

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int, tflite_filename: str):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    best_model.eval().to('cpu')
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution) for _ in range(3))
    onnx_path = os.path.join(output_dir, "tri_left_model.onnx")
    tf_path = os.path.join(output_dir, "tri_left_tf_model")
    tflite_path = os.path.join(output_dir, tflite_filename)

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1: PyTorch -> ONNX
    print("1. Converting to ONNX...")
    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input1', 'input2', 'input3'],
            output_names=['output']
        )
        print(f"   ✅ ONNX saved: {onnx_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 2: ONNX -> TensorFlow
    print("2. ONNX -> TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        if os.path.exists(tf_path):
            import shutil
            shutil.rmtree(tf_path)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TF SavedModel saved: {tf_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 3: TF -> TFLite with SELECT_TF_OPS
    print("3. TF -> TFLite (SELECT_TF_OPS)...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"   ✅ TFLite saved: {tflite_path} ({size_mb:.2f} MB)")
        return tflite_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[1] == 3:  # [B,C,H,W]
            layout = 'NCHW'
        else:  # [B,H,W,C]
            layout = 'NHWC'

    resize_h, resize_w = int(first_shape[1] if layout == 'NHWC' else first_shape[2]), \
                         int(first_shape[2] if layout == 'NHWC' else first_shape[3])

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data['r1'][i], test_data['r2'][i], test_data['r3'][i]]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN TRAINING LOOP
# =========================
if __name__ == "__main__":
    resolution = RESOLUTION
    results = []
    cv_index_records = []

    print(f"\n===== Processing left resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(train_data['label'])

    fold = 1
    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- left Fold {fold} ---")
        cv_index_records.append({"fold": fold, "train_indices": train_idx.tolist(), "val_indices": val_idx.tolist()})

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in train_data.items()}

        train_loader = make_loader(TrileftDataset(train_subset, train_transform), BATCH_CV, True)
        val_loader   = make_loader(TrileftDataset(val_subset,   test_transform),  BATCH_CV, False)

        model = TriResNetleft().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                x1, x2, x3 = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(x1, x2, x3)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{EPOCHS_CV}] Loss: {total_loss:.6f}")

        val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
        result_row = {
            'EyeSet': 'left',
            'Resolution': resolution,
            'Fold': fold,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(result_row)
        print(results)
        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"left_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict()}, fold_path)

        fold += 1

    # Save CV results
    pd.DataFrame(results).to_csv(os.path.join(output_dir, "left_val_cross_validation_results.csv"), index=False)
    with open(os.path.join(output_dir, "left_cv_indices.json"), "w") as f:
        json.dump(cv_index_records, f, indent=2)

    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = df[(df['Val_Precision'] >= 0.90) & (df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1'], ascending=False).iloc[0] if len(candidates) > 0 else \
           df.sort_values(['minPR','Val_F1'], ascending=False).iloc[0]
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best model
    ckpt_path = os.path.join(output_dir, f"left_cv_fold_{best_fold}_res{resolution}.pt")
    best_model = TriResNetleft().to(device)
    state = torch.load(ckpt_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Test evaluation (PyTorch)
    test_loader = make_loader(TrileftDataset(test_data, test_transform), BATCH_CV, False)
    test_metrics = evaluate_with_predictions(best_model, test_loader, test_data['filenames'])

    test_results_df = pd.DataFrame([{
        'ChosenFold': best_fold,
        'Test_Precision': test_metrics[0],
        'Test_Recall': test_metrics[1],
        'Test_F1': test_metrics[2],
        'Test_Accuracy': test_metrics[3],
        'Test_AUC': test_metrics[4],
        'Test_TP': test_metrics[5],
        'Test_TN': test_metrics[6],
        'Test_FP': test_metrics[7],
        'Test_FN': test_metrics[8]
    }])
    test_results_df.to_csv(os.path.join(output_dir, "left_bestfold_test_results.csv"), index=False)

    print("\n📊 TEST Results (Shared Backbone):")
    print(test_results_df.to_string(index=False))

    # Save detailed PyTorch predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(output_dir, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(output_dir, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(output_dir, "confusion_matrix_pytorch.png"))

    # Convert to TFLite
    tflite_filename = "tri_left_eye_resnet18_shared.tflite"
    tflite_path = convert_to_tflite(best_model.cpu(), output_dir, resolution, tflite_filename)

    if tflite_path:
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"\n🎉 SUCCESS! Final TFLite model size: {size_mb:.2f} MB")
        print(f"📍 Path: {tflite_path}")

        # --- 🔍 Re-evaluate TFLite model ---
        print("\n🔍 Re-evaluating TFLite model on test set...")
        try:
            tflite_metrics = evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution)
            tflite_results_df = pd.DataFrame([{
                'Source': 'TFLite',
                'Test_Precision': tflite_metrics[0],
                'Test_Recall': tflite_metrics[1],
                'Test_F1': tflite_metrics[2],
                'Test_Accuracy': tflite_metrics[3],
                'Test_AUC': tflite_metrics[4],
                'Test_TP': tflite_metrics[5],
                'Test_TN': tflite_metrics[6],
                'Test_FP': tflite_metrics[7],
                'Test_FN': tflite_metrics[8]
            }])

            combined = pd.concat([
                test_results_df.assign(Source='PyTorch'),
                tflite_results_df
            ], ignore_index=True)

            print("\n📊 COMPARISON: PyTorch vs TFLite")
            print(combined.to_string(index=False))
            combined.to_csv(os.path.join(output_dir, "pytorch_vs_tflite_comparison.csv"), index=False)

            # Save detailed TFLite predictions to CSV
            save_predictions_to_csv(
                tflite_metrics[11],  # filenames
                tflite_metrics[9],   # true labels
                tflite_metrics[12],  # pred labels
                tflite_metrics[10],  # pred probs
                os.path.join(output_dir, "detailed_predictions_tflite.csv")
            )

            # Plot ROC curve and confusion matrix for TFLite model
            plot_roc_curve(tflite_metrics[9], tflite_metrics[10], 
                           "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                           os.path.join(output_dir, "roc_curve_tflite.png"))
            plot_confusion_matrix(tflite_metrics[9], 
                                  tflite_metrics[12], 
                                  "Confusion Matrix - TFLite Model", 
                                  os.path.join(output_dir, "confusion_matrix_tflite.png"))

            # Create metrics comparison plot
            plot_metrics_comparison(test_results_df.iloc[0].to_dict(), tflite_metrics, 
                                   os.path.join(output_dir, "metrics_comparison.png"))

            tol = 1e-3
            if (abs(tflite_metrics[2] - test_metrics[2]) < tol and
                abs(tflite_metrics[4] - test_metrics[4]) < tol):
                print("✅ TFLite results MATCH PyTorch within tolerance.")
            else:
                print("⚠️ WARNING: TFLite results differ significantly from PyTorch!")
        except Exception as e:
            print(f"❌ TFLite evaluation failed: {e}")
    else:
        print("❌ TFLite conversion failed.")

    print("\n✅ Pipeline completed. Model size reduced to ~45 MB via shared backbone.")
    print(f"✅ Detailed prediction CSVs and plots saved to: {output_dir}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (left)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=44
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=42
left3   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43

📂 TEST  non-anemic (left)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/ | files=314
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_r

INFO:absl:Function `__call__` contains input name(s) input, x, y with unsupported characters which will be renamed to onnx_tf_prefix_identity_49_input, transpose_187_x, onnx_tf_prefix__fusion_fusion_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_tf_model/fingerprint.pb


   ✅ TF SavedModel saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_tf_model
3. TF -> TFLite (SELECT_TF_OPS)...


2025-11-17 18:46:27.155300: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 18:46:27.155354: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 18:46:27.157116: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_tf_model
2025-11-17 18:46:27.195191: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 18:46:27.195228: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_tf_model
2025-11-17 18:46:27.210790: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 18:46:27.257143: 

   ✅ TFLite saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_eye_resnet18_shared.tflite (45.92 MB)

🎉 SUCCESS! Final TFLite model size: 45.92 MB
📍 Path: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/tri_left_eye_resnet18_shared.tflite

🔍 Re-evaluating TFLite model on test set...


2025-11-17 18:46:28.500149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-17 18:46:28.501635: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


🔍 TFLite input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0']
   ➤ Detected layout: NCHW, size: 224x224

📊 COMPARISON: PyTorch vs TFLite
 ChosenFold  Test_Precision  Test_Recall  Test_F1  Test_Accuracy  Test_AUC  Test_TP  Test_TN  Test_FP  Test_FN  Source
        4.0             1.0     0.942857 0.970588       0.993443  0.999577       33      270        0        2 PyTorch
        NaN             1.0     0.942857 0.970588       0.993443  0.999577       33      270        0        2  TFLite
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bestfold_only_shared/detailed_predictions_tflite.csv
✅ TFLite results MATCH PyTorch within tolerance.

✅ Pipeline completed. Model size reduced to ~45 MB via shared backbone.
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye_hb_90_repro_bes

In [9]:
# -*- coding: utf-8 -*-
"""
Reproducible Hexa-Eye ResNet18 with SHARED BACKBONE + EARLY STOPPING & TFLite Verification
----------------------------------------------------------------------------------
✅ Single ResNet18 used across 6 inputs → ~47 MB TFLite
✅ Stops fold early if Val Precision & Recall >= 0.90
✅ Re-evaluates .tflite model and compares predictions with PyTorch
✅ Ensures faithful deployment (no numerical drift)
✅ Deterministic training + FlexDelegate support for GELU
✅ Added detailed prediction CSV with file IDs, predictions, probabilities, and confusion matrix indicators
✅ Added comprehensive plotting: ROC curves, confusion matrices, metrics comparison
"""

import os
import json
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224
EPOCHS_CV = 300
BATCH_CV = 8  # Reduced due to 6 inputs per sample
LR_CV = 0.000256


# 🔥 NEW: Early stop if both P and R >= this threshold
EARLY_STOP_PR = 0.90

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

os.environ["PYTHONHASHSEED"] = str(SEED)
set_global_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "hexa_eye_hb_90_repro_bestfold_only_shared2")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

# All six input directories
train_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS
# =========================
def base_from(fname, suffix):
    return fname[:-len(suffix)] if fname.endswith(suffix) else None

def common_bases_hexa(dirs_map):
    suffixes = {
        'left1': '_left_eye_1.png', 'left2': '_left_eye_2.png', 'left3': '_left_eye_3.png',
        'right1': '_right_eye_1.png', 'right2': '_right_eye_2.png', 'right3': '_right_eye_3.png'
    }
    bases_sets = []
    for k in dirs_map.keys():
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            return []
        names = [f for f in os.listdir(folder) if f.endswith(suffixes[k])]
        bases = {base_from(f, suffixes[k]) for f in names if base_from(f, suffixes[k]) is not None}
        bases_sets.append(bases)
    if not bases_sets:
        return []
    inter = set.intersection(*bases_sets)
    return sorted(inter)

def load_hexa_images_by_bases_with_filenames(dirs_map, bases):
    out = {f'r{i}': [] for i in range(1,7)}
    out['filenames'] = []
    key_map = [
        ('left1', '_left_eye_1.png'),
        ('left2', '_left_eye_2.png'),
        ('left3', '_left_eye_3.png'),
        ('right1', '_right_eye_1.png'),
        ('right2', '_right_eye_2.png'),
        ('right3', '_right_eye_3.png')
    ]
    for b in bases:
        imgs, failed = [], False
        for long_k, suf in key_map:
            path = os.path.join(dirs_map[long_k], b + suf)
            if not os.path.isfile(path):
                failed = True
                break
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not failed:
            for i, img in enumerate(imgs):
                out[f'r{i+1}'].append(img)
            out['filenames'].append(b)  # Use base name as identifier
    return out

def prepare_dataset_hexa_with_filenames(anemic_dirs, non_dirs, split_name="(split)"):
    bases_a = common_bases_hexa(anemic_dirs)
    bases_n = common_bases_hexa(non_dirs)
    imgs_a = load_hexa_images_by_bases_with_filenames(anemic_dirs, bases_a)
    imgs_n = load_hexa_images_by_bases_with_filenames(non_dirs, bases_n)
    
    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'r4': imgs_a['r4'] + imgs_n['r4'],
        'r5': imgs_a['r5'] + imgs_n['r5'],
        'r6': imgs_a['r6'] + imgs_n['r6'],
        'filenames': imgs_a['filenames'] + imgs_n['filenames'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }
    print(f"✅ {split_name}: anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    try:
        df_bases = pd.DataFrame({'class': ['anemic']*len(bases_a) + ['non_anemic']*len(bases_n),
                                 'base_id': bases_a + bases_n})
        df_bases.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_hexa.csv"), index=False)
    except Exception:
        pass
    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in dirs_map.keys():
        p = dirs_map[k]; c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# LOAD DATA
# =========================

print_dir_stats("TEST  anemic (HEXA)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (HEXA)", test_dirs_non)

train_data = prepare_dataset_hexa_with_filenames(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_hexa_with_filenames(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_hexa_with_filenames(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No hexa-eye TRAIN samples found.")

# =========================
# DATASET
# =========================
class HexaDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
    def __len__(self):
        return len(self.data['label'])
    def __getitem__(self, idx):
        images = [self.data[f'r{i}'][idx] for i in range(1,7)]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
    )

# =========================
# MODEL: SHARED RESNET18 BACKBONE (6 inputs)
# =========================
class HexaResNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Single shared backbone
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Identity()

        # Fusion head: 6*512 → 1
        self.fusion = nn.Sequential(
            nn.Linear(6 * 512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, x1, x2, x3, x4, x5, x6):
        f1 = self.backbone(x1)
        f2 = self.backbone(x2)
        f3 = self.backbone(x3)
        f4 = self.backbone(x4)
        f5 = self.backbone(x5)
        f6 = self.backbone(x6)
        x = torch.cat([f1, f2, f3, f4, f5, f6], dim=1)
        return self.fusion(x)

# =========================
# EVALUATION (PyTorch) WITH PREDICTIONS
# =========================
@torch.no_grad()
def evaluate_with_predictions(model, loader, filenames):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    all_filenames = []
    
    # Get all filenames in loader order
    batch_size = loader.batch_size
    for i in range(0, len(filenames), batch_size):
        batch_end = min(i + batch_size, len(filenames))
        all_filenames.extend(filenames[i:batch_end])
    
    for imgs, labels in loader:
        x_list = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(*x_list)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(all_labels) == 0 or len(set(all_labels)) < 2:
        return [float('nan')] * 9 + [all_labels, all_probs, all_filenames, all_preds]

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# TFLITE CONVERSION
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int, tflite_filename: str):
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    best_model.eval().to('cpu')
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution) for _ in range(6))
    onnx_path = os.path.join(output_dir, "hexa_model.onnx")
    tf_path = os.path.join(output_dir, "hexa_tf_model")
    tflite_path = os.path.join(output_dir, tflite_filename)

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1: PyTorch -> ONNX
    print("1. Converting to ONNX...")
    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=[f'input{i}' for i in range(1,7)],
            output_names=['output']
        )
        print(f"   ✅ ONNX saved: {onnx_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 2: ONNX -> TensorFlow
    print("2. ONNX -> TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        if os.path.exists(tf_path):
            import shutil
            shutil.rmtree(tf_path)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TF SavedModel saved: {tf_path}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Step 3: TF -> TFLite with SELECT_TF_OPS
    print("3. TF -> TFLite (SELECT_TF_OPS)...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"   ✅ TFLite saved: {tflite_path} ({size_mb:.2f} MB)")
        return tflite_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# =========================
# TFLITE RE-EVALUATION WITH PREDICTIONS
# =========================
def evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution):
    import tensorflow as tf
    from PIL import Image
    import numpy as np
    from torchvision.transforms.functional import to_tensor, normalize

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Sort by name to ensure correct order
    sorted_inputs = sorted(input_details, key=lambda x: x['name'])
    print(f"🔍 TFLite input names: {[d['name'] for d in sorted_inputs]}")

    first_shape = sorted_inputs[0]['shape']
    if len(first_shape) == 4:
        if first_shape[1] == 3:  # [B,C,H,W]
            layout = 'NCHW'
        else:  # [B,H,W,C]
            layout = 'NHWC'

    resize_h, resize_w = int(first_shape[1] if layout == 'NHWC' else first_shape[2]), \
                         int(first_shape[2] if layout == 'NHWC' else first_shape[3])

    print(f"   ➤ Detected layout: {layout}, size: {resize_h}x{resize_w}")

    def preprocess_pil_style(img_rgb):
        img_pil = Image.fromarray(img_rgb)
        img_resized = img_pil.resize((resize_w, resize_h), Image.BILINEAR)
        tensor = to_tensor(img_resized)
        normalized = normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        return normalized.numpy()

    all_preds, all_probs, all_labels, all_filenames = [], [], [], test_data['filenames']

    for i in range(len(test_data['label'])):
        imgs = [test_data[f'r{j}'][i] for j in range(1,7)]
        label = test_data['label'][i]

        for idx, detail in enumerate(sorted_inputs):
            raw_img = imgs[idx]
            processed = preprocess_pil_style(raw_img)

            if layout == 'NCHW':
                model_input = np.expand_dims(processed, axis=0).astype(detail['dtype'])
            else:
                nhwc = np.transpose(processed, (1, 2, 0))
                model_input = np.expand_dims(nhwc, axis=0).astype(detail['dtype'])

            interpreter.set_tensor(detail['index'], model_input)

        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        logit = float(np.array(output).reshape(-1)[0])
        prob = 1.0 / (1.0 + np.exp(-logit))
        pred = int(prob > 0.5)

        all_preds.append(pred)
        all_probs.append(prob)
        all_labels.append(label)

    if len(set(all_labels)) < 2:
        auc = float('nan')
    else:
        auc = roc_auc_score(all_labels, all_probs)

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()

    return p, r, f1, acc, auc, tp, tn, fp, fn, all_labels, all_probs, all_filenames, all_preds

# =========================
# PLOTTING FUNCTIONS
# =========================
def plot_roc_curve(y_true, y_scores, title, save_path):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, auc
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrix(y_true, y_pred, title, save_path):
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Anemic', 'Anemic'],
                yticklabels=['Non-Anemic', 'Anemic'])
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(pytorch_metrics, tflite_metrics, save_path):
    import matplotlib.pyplot as plt
    
    metrics = ['Precision', 'Recall', 'F1-Score', 'Accuracy', 'AUC']
    pytorch_vals = [pytorch_metrics['Test_Precision'], pytorch_metrics['Test_Recall'], 
                    pytorch_metrics['Test_F1'], pytorch_metrics['Test_Accuracy'], 
                    pytorch_metrics['Test_AUC']]
    tflite_vals = [tflite_metrics[0], tflite_metrics[1], 
                   tflite_metrics[2], tflite_metrics[3], 
                   tflite_metrics[4]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, pytorch_vals, width, label='PyTorch', color='steelblue')
    plt.bar(x + width/2, tflite_vals, width, label='TFLite', color='darkorange')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('PyTorch vs TFLite Performance Comparison')
    plt.xticks(x, metrics)
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

# =========================
# SAVE PREDICTIONS TO CSV
# =========================
def save_predictions_to_csv(filenames, true_labels, pred_labels, pred_probs, output_path):
    # Convert labels to readable format
    true_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in true_labels]
    pred_labels_str = ['Anemic' if label == 1 else 'Non-Anemic' for label in pred_labels]
    
    # Calculate confusion matrix indicators
    tp = [1 if (t == 1 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    tn = [1 if (t == 0 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    fp = [1 if (t == 0 and p == 1) else 0 for t, p in zip(true_labels, pred_labels)]
    fn = [1 if (t == 1 and p == 0) else 0 for t, p in zip(true_labels, pred_labels)]
    
    # Create DataFrame
    df = pd.DataFrame({
        'file_id': filenames,
        'actual_value': true_labels_str,
        'predicted_value': pred_labels_str,
        'predicted_probability': pred_probs,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn
    })
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to: {output_path}")

# =========================
# MAIN TRAINING LOOP (with EARLY STOPPING)
# =========================
if __name__ == "__main__":
    resolution = RESOLUTION
    results = []
    cv_index_records = []

    print(f"\n===== Processing HEXA resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(train_data['label'])

    fold = 1
    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- HEXA Fold {fold} ---")
        cv_index_records.append({"fold": fold, "train_indices": train_idx.tolist(), "val_indices": val_idx.tolist()})

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in train_data.items()}

        train_loader = make_loader(HexaDataset(train_subset, train_transform), BATCH_CV, True)
        val_loader   = make_loader(HexaDataset(val_subset,   test_transform),  BATCH_CV, False)

        model = HexaResNet().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        stopped_early = False
        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                x_list = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(*x_list)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{EPOCHS_CV}] Loss: {total_loss:.6f}")

            # 🔥 EARLY STOPPING CHECK
            if EARLY_STOP_PR is not None:
                val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
                p, r = val_metrics[0], val_metrics[1]
                if not (np.isnan(p) or np.isnan(r)) and p >= EARLY_STOP_PR and r >= EARLY_STOP_PR and p < 1 and r < 1:
                    print(f"✅ Early stop at epoch {epoch+1}: P={p:.3f}, R={r:.3f}")
                    stopped_early = True
                    break  # exit inner loop

        # Final evaluation after training (or early stop)
        val_metrics = evaluate_with_predictions(model, val_loader, val_subset['filenames'])
        result_row = {
            'EyeSet': 'HEXA',
            'Resolution': resolution,
            'Fold': fold,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8],
            'Stopped_Early': stopped_early
        }
        results.append(result_row)
        print(results)
        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"hexa_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict()}, fold_path)

        fold += 1

    # Save CV results
    pd.DataFrame(results).to_csv(os.path.join(output_dir, "hexa_val_cross_validation_results.csv"), index=False)
    with open(os.path.join(output_dir, "hexa_cv_indices.json"), "w") as f:
        json.dump(cv_index_records, f, indent=2)

    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = df[(df['Val_Precision'] >= 0.90) & (df['Val_Recall'] >= 0.90) & (df['Val_Precision'] < 1) & (df['Val_Recall'] < 1)]
    best = candidates.sort_values(['Val_F1'], ascending=False).iloc[0] if len(candidates) > 0 else \
           df.sort_values(['minPR','Val_F1'], ascending=False).iloc[0]
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best model
    ckpt_path = os.path.join(output_dir, f"hexa_cv_fold_{best_fold}_res{resolution}.pt")
    best_model = HexaResNet().to(device)
    state = torch.load(ckpt_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Test evaluation
    test_loader = make_loader(HexaDataset(test_data, test_transform), BATCH_CV, False)
    test_metrics = evaluate_with_predictions(best_model, test_loader, test_data['filenames'])

    test_results_df = pd.DataFrame([{
        'ChosenFold': best_fold,
        'Test_Precision': test_metrics[0],
        'Test_Recall': test_metrics[1],
        'Test_F1': test_metrics[2],
        'Test_Accuracy': test_metrics[3],
        'Test_AUC': test_metrics[4],
        'Test_TP': test_metrics[5],
        'Test_TN': test_metrics[6],
        'Test_FP': test_metrics[7],
        'Test_FN': test_metrics[8]
    }])
    test_results_df.to_csv(os.path.join(output_dir, "hexa_bestfold_test_results.csv"), index=False)

    print("\n📊 TEST Results (Shared Backbone):")
    print(test_results_df.to_string(index=False))

    # Save detailed PyTorch predictions to CSV
    save_predictions_to_csv(
        test_metrics[11],  # filenames
        test_metrics[9],   # true labels
        test_metrics[12],  # pred labels
        test_metrics[10],  # pred probs
        os.path.join(output_dir, "detailed_predictions_pytorch.csv")
    )

    # Plot ROC curve and confusion matrix for PyTorch model
    plot_roc_curve(test_metrics[9], test_metrics[10], 
                   "ROC Curve - PyTorch Model (Original Chronological Test Set)", 
                   os.path.join(output_dir, "roc_curve_pytorch.png"))
    plot_confusion_matrix(test_metrics[9], 
                          test_metrics[12], 
                          "Confusion Matrix - PyTorch Model", 
                          os.path.join(output_dir, "confusion_matrix_pytorch.png"))

    # Convert to TFLite
    tflite_filename = "hexa_eye_resnet18_shared.tflite"
    tflite_path = convert_to_tflite(best_model.cpu(), output_dir, resolution, tflite_filename)

    if tflite_path:
        size_mb = os.path.getsize(tflite_path) / (1024 * 1024)
        print(f"\n🎉 SUCCESS! Final TFLite model size: {size_mb:.2f} MB")
        print(f"📍 Path: {tflite_path}")

        # --- 🔍 Re-evaluate TFLite model ---
        print("\n🔍 Re-evaluating TFLite model on test set...")
        try:
            tflite_metrics = evaluate_tflite_model_with_predictions(tflite_path, test_data, resolution)
            tflite_results_df = pd.DataFrame([{
                'Source': 'TFLite',
                'Test_Precision': tflite_metrics[0],
                'Test_Recall': tflite_metrics[1],
                'Test_F1': tflite_metrics[2],
                'Test_Accuracy': tflite_metrics[3],
                'Test_AUC': tflite_metrics[4],
                'Test_TP': tflite_metrics[5],
                'Test_TN': tflite_metrics[6],
                'Test_FP': tflite_metrics[7],
                'Test_FN': tflite_metrics[8]
            }])

            combined = pd.concat([
                test_results_df.assign(Source='PyTorch'),
                tflite_results_df
            ], ignore_index=True)

            print("\n📊 COMPARISON: PyTorch vs TFLite")
            print(combined.to_string(index=False))
            combined.to_csv(os.path.join(output_dir, "pytorch_vs_tflite_comparison.csv"), index=False)

            # Save detailed TFLite predictions to CSV
            save_predictions_to_csv(
                tflite_metrics[11],  # filenames
                tflite_metrics[9],   # true labels
                tflite_metrics[12],  # pred labels
                tflite_metrics[10],  # pred probs
                os.path.join(output_dir, "detailed_predictions_tflite.csv")
            )

            # Plot ROC curve and confusion matrix for TFLite model
            plot_roc_curve(tflite_metrics[9], tflite_metrics[10], 
                           "ROC Curve - TFLite Model (Original Chronological Test Set)", 
                           os.path.join(output_dir, "roc_curve_tflite.png"))
            plot_confusion_matrix(tflite_metrics[9], 
                                  tflite_metrics[12], 
                                  "Confusion Matrix - TFLite Model", 
                                  os.path.join(output_dir, "confusion_matrix_tflite.png"))

            # Create metrics comparison plot
            plot_metrics_comparison(test_results_df.iloc[0].to_dict(), tflite_metrics, 
                                   os.path.join(output_dir, "metrics_comparison.png"))

            tol = 1e-3
            if (abs(tflite_metrics[2] - test_metrics[2]) < tol and
                abs(tflite_metrics[4] - test_metrics[4]) < tol):
                print("✅ TFLite results MATCH PyTorch within tolerance.")
            else:
                print("⚠️ WARNING: TFLite results differ significantly from PyTorch!")
        except Exception as e:
            print(f"❌ TFLite evaluation failed: {e}")
    else:
        print("❌ TFLite conversion failed.")

    print("\n✅ Hexa-Eye pipeline completed. Model size remains ~47 MB thanks to shared backbone.")
    print(f"✅ Detailed prediction CSVs and plots saved to: {output_dir}")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TEST  anemic (HEXA)
left1   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=44
left2   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=42
left3   | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=43
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/ | files=40
right3  | /home/u

INFO:absl:Function `__call__` contains input name(s) input, x, y with unsupported characters which will be renamed to onnx_tf_prefix_identity_49_input, transpose_373_x, onnx_tf_prefix__fusion_fusion_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_tf_model/fingerprint.pb


   ✅ TF SavedModel saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_tf_model
3. TF -> TFLite (SELECT_TF_OPS)...


2025-11-17 23:47:04.078876: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-11-17 23:47:04.078912: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-11-17 23:47:04.081196: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_tf_model
2025-11-17 23:47:04.119885: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-11-17 23:47:04.119904: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_tf_model
2025-11-17 23:47:04.141262: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-11-17 23:47:04.218146: I tensorflow/c

   ✅ TFLite saved: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_eye_resnet18_shared.tflite (48.96 MB)

🎉 SUCCESS! Final TFLite model size: 48.96 MB
📍 Path: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/hexa_eye_resnet18_shared.tflite

🔍 Re-evaluating TFLite model on test set...


2025-11-17 23:47:05.908276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-17 23:47:05.909611: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


🔍 TFLite input names: ['serving_default_input1:0', 'serving_default_input2:0', 'serving_default_input3:0', 'serving_default_input4:0', 'serving_default_input5:0', 'serving_default_input6:0']
   ➤ Detected layout: NCHW, size: 224x224

📊 COMPARISON: PyTorch vs TFLite
 ChosenFold  Test_Precision  Test_Recall  Test_F1  Test_Accuracy  Test_AUC  Test_TP  Test_TN  Test_FP  Test_FN  Source
        4.0             1.0     0.964286 0.981818       0.996226       1.0       27      237        0        1 PyTorch
        NaN             1.0     0.964286 0.981818       0.996226       1.0       27      237        0        1  TFLite
✅ Predictions saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hexa_eye_hb_90_repro_bestfold_only_shared2/detailed_predictions_tflite.csv
✅ TFLite results MATCH PyTorch within tolerance.

✅ Hexa-Eye pipeline completed. Model size remains ~47 MB thanks to shared backbone.
✅ Detailed prediction CSVs and plots saved to: /home/ubuntu/ane